# Chapter 8 - 가우스 소거법

이 장에서 $\mathbb R$상의 행렬에 적용되는 과정의 개요를 설명하지만, 주로 GF(2)상의 행렬에 적용되는 가우스 소거법에 중점을 둔다. 이아고리즘은 다음에 적용된다.
- 주어진 벡터들의 생성에 대한 기저 찾기. 이것은 또한 랭크에 대한 알고리즘을 제공하여 일차종속을 테스트할 수도 있다.
- 행렬의 영공간에 대한 기저 찾기.
- 행렬방적ㅇ식의 해 구하기. 이것은 주어진 벡터를 다른 주어진 벡터들의 선형결합으로 표현하는 것과 동일하다. 또한, 이것은 선형방정식들의 시스템에 대한 해를 구하는 것과 동일하다.

# 8.1 사다리꼴(Echelon from)

### Definition 8.1.1
m x n행렬 A는 만약 다음 조건을 만족하면 *사다리꼴*이다. 즉, 임의의 행에 대해, 만약 그 행의 첫 번째 영이 아닌 엔트리가 위치 k에 있으면 그 행 이전의 모든 행의 첫 번째 영이 아닌 엔트리는 k보다 작은 어떤 위치에 있다.

## 8.1.1 사다리꼴에서 행공간에 대한 기저로

### Lemma 8.1.2
만약 어떤 행렬이 사다리꼴이면, 영이 아닌 행들은 행공간에 대한 기저를 형성한다.

## 8.1.2 사다리꼴 행렬의 행리스트

행렬의 사다리꼴은 행과 관련된 것이 많으며, 행렬을 Mat의 인스턴스가 아니라 행 리스트, 즉 벡터들의 리스트로 나타낼 때 사다리꼴을 이용하면 편리하다.

## 8.1.3 맨 왼쪽의 영이 아닌 위치에 의한 행들의 정렬

In [41]:
from vecutil import list2vec

# example mat
rowlist = [list2vec(v) for v in [[0,2,3,4,5],[0,0,0,3,2],[1,2,3,4,5],[0,0,0,6,7],[0,0,0,9,9]]]

# sort column labels
col_label_list = sorted(rowlist[0].D, key=hash)

# sort to echelon form
new_rowlist = []
row_left = set(range(len(rowlist)))

for c in col_label_list:
    rows_with_nonzeros = [r for r in row_left if rowlist[r][c] != 0]
    if rows_with_nonzeros != []:
        pivot = rows_with_nonzeros[0]
        new_rowlist.append(rowlist[pivot])
        row_left.remove(pivot)
        
new_rowlist

[Vec({0, 1, 2, 3, 4},{0: 1, 1: 2, 2: 3, 3: 4, 4: 5}),
 Vec({0, 1, 2, 3, 4},{0: 0, 1: 2, 2: 3, 3: 4, 4: 5}),
 Vec({0, 1, 2, 3, 4},{0: 0, 1: 0, 2: 0, 3: 3, 4: 2}),
 Vec({0, 1, 2, 3, 4},{0: 0, 1: 0, 2: 0, 3: 6, 4: 7})]

new_rowlist에 추가된 행은 피봇행(pivot row)이라 하고, 열 c의 피봇행의 원소는 피봇원소(pivot element)라고 한다.
___

위의 결과는 사라리꼴 정의에 위배된다. 네 번째 행의 첫 번째 영이 아닌 엔트리는 네 번째 위치에 있으므로, 모든 이전 행의 첫 번째 영이 아닌 엔트리는 세 번째 위치의 왼쪽에 있어ㅑ 한다. 하지만, 세 번째 행의 첫 번째 영이 아닌 엔트리는 네 번째 위치에 있다.

## 8.1.4 기본행덧셈 연산

8.1.3 알고리즘은 기본행덧셈(elementary row-addition operation)을 수행하여 피봇행이 아닌 다른 행에 있는 영이 아닌 원소를 영으로 만든다.

In [46]:
from vecutil import list2vec

# example mat
rowlist = [list2vec(v) for v in [[0,2,3,4,5],[0,0,0,3,2],[1,2,3,4,5],[0,0,0,6,7],[0,0,0,9,9]]]

# sort column labels
col_label_list = sorted(rowlist[0].D, key=hash)

# sort to echelon form
new_rowlist = []
row_left = set(range(len(rowlist)))

for c in col_label_list:
    rows_with_nonzeros = [r for r in row_left if rowlist[r][c] != 0]
    if rows_with_nonzeros != []:
        pivot = rows_with_nonzeros[0]
        new_rowlist.append(rowlist[pivot])
        row_left.remove(pivot)
        
        # elementary row-addition operation
        for r in rows_with_nonzeros[1:]:
            multiplier = rowlist[r][c]/rowlist[pivot][c]
            rowlist[r] -= multiplier*rowlist[pivot]
        
        
new_rowlist

[Vec({0, 1, 2, 3, 4},{0: 1, 1: 2, 2: 3, 3: 4, 4: 5}),
 Vec({0, 1, 2, 3, 4},{0: 0, 1: 2, 2: 3, 3: 4, 4: 5}),
 Vec({0, 1, 2, 3, 4},{0: 0, 1: 0, 2: 0, 3: 3, 4: 2}),
 Vec({0, 1, 2, 3, 4},{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 3.0})]

## 8.1.5 기본행덧셈 행렬에 의한 곱셈

한 행의 배수를 다른 행에서 빼는 것은 그 행렬을 기본행덕셈 행렬과 곱합으로써 이룰어질 수 있다. 5장에 살펴보았듯이, 이러한 행렬은 가역적이다.

$\begin{bmatrix} 1 && 0 && 0 && 0  \\ 0 && 1 && 0 && 0 \\ 0 && 0 && 1 && 0 \\ 0 && 0 && -2 && 1 \end{bmatrix} \begin{bmatrix} 1 && 2 && 3 && 4 && 5 \\ 0 && 2 && 3 && 4 && 5 \\ 0 && 0 && 0 && 3 && 2 \\ 0 && 0 && 0 && 6 && 7 \end{bmatrix} = \begin{bmatrix} 1 && 2 && 3 && 4 && 5 \\ 0 && 2 && 3 && 4 && 5 \\ 0 && 0 && 0 && 3 && 2 \\ 0 && 0 && 0 && 0 && 3 \end{bmatrix}$

## 8.1.6 행덧셈 연산은 행공간을 유지한다.

행렬을 사다리꼴로 변환하는 명목상의 목적은 그 행렬의 행공간에 대한 기저를 얻는 것이다.

### Lemma 8.1.3
행렬 A와 N에 대해, $RowNA\subseteq RowA$이다. 

### Corollary 8.1.4
행렬 A와 M에 대해, 만약 M이 가역적이면 $RowMA=RowA$이다.

### Example 8.1.5
섹션 8.1.4의 예들 다시 살펴보자. Lemma 8.1.3의 주장을 사용하여 $RowMA\subseteq A$이고 $RowA\subseteq RowMA$임을 확인해보자.

- $RowMA$에 속하는 모든 벡터 v는 벡터와 행렬 A의 곱셈으로 표현될 수 있음을 보여준다. 이것은 또한 v가 RowA임을 보여준다.
- A는 $M^{-1}MA$로 표현되고 $M^{-1}MA$에 속하는 모든 벡터 v는 벡터와 행렬 MA의 곱셈으로 표현될 수 있음을 보여준다. 이것은 또한 v가 $RowMA$에 속한다는 것을 보여 준다.

## 8.1.7 가우스 소거법을 통한 기저, 랭크, 일차독립

작성한 프로그램은 프로시저, row_reduce(rowlist)에 포함되었다. 

주어진 벡터들의 생성에 대한 기저를 찾는 프로시저를 가지고 있으므로, 랭크 및 일차독립에 대한 프로시저는 쉽게 작성할 수 있다.

In [50]:
from echelon import row_reduce

rowlist = [list2vec(v) for v in [[0,2,3,4,5],[0,0,0,3,2],[1,2,3,4,5],[0,0,0,6,7],[0,0,0,9,9]]]
row_reduce(rowlist)

[Vec({0, 1, 2, 3, 4},{0: 1, 1: 2, 2: 3, 3: 4, 4: 5}),
 Vec({0, 1, 2, 3, 4},{0: 0, 1: 2, 2: 3, 3: 4, 4: 5}),
 Vec({0, 1, 2, 3, 4},{0: 0, 1: 0, 2: 0, 3: 3, 4: 2}),
 Vec({0, 1, 2, 3, 4},{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 3.0})]

## 8.1.8 가우스 소거법이 실패할 때

파이썬은 부동소수를 사용하여 계산을 수행하고 산술연산은 근사적으로 옳다. 따라서 파이썬의 계산 결과를 벡터 집합의 랭크를 결정하는 데 사용하기 어려울 수 있다.

In [67]:
1 - 1e20

-1e+20

In [66]:
1 - 1e20  + 1e20

0.0

## 8.1.9 피봇팅 및 수치해석

*피봇팅(pivoting)*은 피봇원소를 주의깊게 선택하는 것을 말한다. 여기에는 두 가지 전략이 사용된다.
- 부분적 피봇팅(partial pivoting): 열 c에 영이 아닌 엔트리를 가지는 행 중에서 절대값이 가장 큰 엔트리를 가지는 행을 선택한다.
- 완전 피봇팅(complete pivoting): 미리 열의 순서를 선택하는 대신에 그때그때 피봇원소를 최대로 하는 각 열을 선택한다.

보통은 구현하기 쉽고 실행속도가 빠르기 때문에 부분적 피봇팅이 실직적으로 사용된다. 이론적으로는 여전히 아주 큰 행렬에 대해 심각하게 나쁜 결과를 초래할 수 있다.

정확하지 않은 연산을 사용하여 행렬의 랭크를 계산하는 것은 아주 어렵다. 일반적으로 인정되는 방법은 행렬의 특이값분해(singular value decomposition)를 사용한다.

In [69]:
1 + 1e20

1e+20

In [70]:
1 + 1e20 - 1e20

0.0

# 8.2 GF(2)상의 가우스 소거법

가우스 소거법은 GF(2)상의 벡터들에 대해 수행될 수 있다. 이 경우, 모든 연산은 정확하고 수치적 문제를 발생하지 않는다.

In [72]:
from vecutil import list2vec
from GF2 import one

# example mat
rowlist = [list2vec(v) for v in [[0,0,one,one],[one,0,one,one],[one,0,0,one],[one,one,one,one]]]

# sort column labels
col_label_list = sorted(rowlist[0].D, key=hash)

# sort to echelon form
new_rowlist = []
row_left = set(range(len(rowlist)))

for c in col_label_list:
    rows_with_nonzeros = [r for r in row_left if rowlist[r][c] != 0]
    if rows_with_nonzeros != []:
        pivot = rows_with_nonzeros[0]
        new_rowlist.append(rowlist[pivot])
        row_left.remove(pivot)
        
        # elementary row-addition operation
        for r in rows_with_nonzeros[1:]:
            multiplier = rowlist[r][c]/rowlist[pivot][c]
            rowlist[r] -= multiplier*rowlist[pivot]
        
        
new_rowlist

[Vec({0, 1, 2, 3},{0: one, 1: 0, 2: one, 3: one}),
 Vec({0, 1, 2, 3},{0: 0, 1: one, 2: 0, 3: 0}),
 Vec({0, 1, 2, 3},{0: 0, 1: 0, 2: one, 3: one}),
 Vec({0, 1, 2, 3},{0: 0, 1: 0, 2: 0, 3: one})]

# 8.3 다른 문제에 대해 가우스 소거법 사용하기

가우스 소거법은 행렬의 행공간에 대한 기저를 찾는 문제 이외에 다른 문제들을 해결하는 데도 사용될 수 있다.
- 선형시스템의 해 구하기
- 영공간에 대한 기저 찾기

## 8.3.1 가역행렬 M과 사다리꼴의 행렬 MA

가우스 소거법을 다른 문제들을 해결하는 데 사용하는 핵심은 입력 행렬을 사다리꼴 행렬로 만드는데 사용되는 기본행-덧셈 연산을 추적하는 것이다.

### Proposition 8.3.1
임의의 행렬 A에 대해, MA는 사다리꼴 행렬인 그러한 가역 행렬 M이 있다.

## 8.3.2 행렬 곱셈없이 M 계산하기

M을 계산하는 프로시저는 아래의 rowlist에 의해 표현되는 두 개의 행렬을 유지한다.
- 코드에서 rowlist에 의해 표현되는 변환되는 행렬
- 코드에서 M_rowlist에 의해 표현되는 변환하는 행렬
 
알고리즘은 변환하는 행렬과 입력 행렬의 곱이 rowlist에 의해 표현되는 행렬과 동일하게 되도록 유지한다.
- $M\_rowlist(초기행렬)=rowlist$

i번째 행-덧셈 연산을 수행하는 것은 rowlist의 어느 한 행의 배수를 또 다른 행에서 빼는 것으로 구성된다. 이것은 행렬 rowlist를 행-덧셈 행렬 $M_i$와 곱하는 것과 동일하다.
- $M_i(M\_rowlist)(초기행렬)=M_i(rowlist)$

### Example 8.3.2
다음 행렬을 사용하여 예를 실행해 보자.
$A=\begin{bmatrix} 0 && 2 && 3 && 4 && 5 \\ 0 && 0 && 0 && 3 && 2 \\ 1 && 2 && 3 && 4 && 5 \\ 0 && 0 && 0 && 6 && 7 \\ 0 && 0 && 0 && 9 && 8\end{bmatrix}$

...

따라서, 식은 다음과 같이 된다.
$\begin{bmatrix} 1 &&  &&  &&  &&  \\  && 1 &&  &&  &&  \\  &&  && 1 &&  &&  \\  && -2 &&  && 1 &&  \\  && -1\frac{1}{3} &&  && -\frac{2}{3} && 1\end{bmatrix}\begin{bmatrix} 0 && 2 && 3 && 4 && 5 \\ 0 && 0 && 0 && 3 && 2 \\ 1 && 2 && 3 && 4 && 5 \\ 0 && 0 && 0 && 6 && 7 \\ 0 && 0 && 0 && 9 && 8\end{bmatrix}=\begin{bmatrix} 0 && 2 && 3 && 4 && 5 \\ 0 && 0 && 0 && 3 && 2 \\ 1 && 2 && 3 && 4 && 5 \\ 0 && 0 && 0 && 0 && 3 \\ 0 && 0 && 0 && 0 && 0\end{bmatrix}$

In [2]:
from vec import Vec
from vecutil import list2vec

# example mat
rowlist = [list2vec(v) for v in [[0,2,3,4,5],[0,0,0,3,2],[1,2,3,4,5],[0,0,0,6,7],[0,0,0,9,8]]]

# sort column labels
col_label_list = sorted(rowlist[0].D, key=hash)
row_label_list = col_label_list

# sort to echelon form
M_rowlist = [Vec(rowlist[0].D, {row_label_list[i]:1}) for i in range(len(rowlist))]
new_M_rowlist = []
row_left = set(range(len(rowlist)))

for c in col_label_list:
    rows_with_nonzeros = [r for r in row_left if rowlist[r][c] != 0]
    if rows_with_nonzeros != []:
        pivot = rows_with_nonzeros[0]
        new_M_rowlist.append(M_rowlist[pivot])
        row_left.remove(pivot)
        
        # elementary row-addition operation
        for r in rows_with_nonzeros[1:]:
            multiplier = rowlist[r][c]/rowlist[pivot][c]
            rowlist[r] -= multiplier*rowlist[pivot]
            M_rowlist[r] -= multiplier*M_rowlist[pivot]

for r in row_left: new_M_rowlist.append(M_rowlist[r])
        
new_M_rowlist

[Vec({0, 1, 2, 3, 4},{2: 1}),
 Vec({0, 1, 2, 3, 4},{0: 1}),
 Vec({0, 1, 2, 3, 4},{1: 1}),
 Vec({0, 1, 2, 3, 4},{0: 0, 1: -2.0, 2: 0, 3: 1, 4: 0}),
 Vec({0, 1, 2, 3, 4},{0: 0.0, 1: -1.6666666666666667, 2: 0.0, 3: -0.6666666666666666, 4: 1.0})]

In [6]:
from echelon import transformation
from matutil import listlist2mat

A = listlist2mat([[0,2,3,4,5],[0,0,0,3,2],[1,2,3,4,5],[0,0,0,6,7],[0,0,0,9,8]])
M = transformation(A, is_GF2=False)
print('A', A)
print('M', M)
print('MA', M*A)

A 
       0 1 2 3 4
     -----------
 0  |  0 2 3 4 5
 1  |  0 0 0 3 2
 2  |  1 2 3 4 5
 3  |  0 0 0 6 7
 4  |  0 0 0 9 8

M 
       0     1 2      3 4
     --------------------
 0  |  0     0 1      0 0
 1  |  1     0 0      0 0
 2  |  0     1 0      0 0
 3  |  0    -2 0      1 0
 4  |  0 -1.67 0 -0.667 1

MA 
       0 1 2 3 4
     -----------
 0  |  1 2 3 4 5
 1  |  0 2 3 4 5
 2  |  0 0 0 3 2
 3  |  0 0 0 0 3
 4  |  0 0 0 0 0



### Example 8.3.3 
변환하는 행렬을 유지하는 또 다른 예가 있다.

In [5]:
A = listlist2mat([[0,2,4,2,8],[2,1,0,5,4],[4,1,2,4,2],[5,0,0,2,8]])
M = transformation(A, is_GF2=False)
print('A', A)
print('M', M)
print('MA', M*A)

A 
       0 1 2 3 4
     -----------
 0  |  0 2 4 2 8
 1  |  2 1 0 5 4
 2  |  4 1 2 4 2
 3  |  5 0 0 2 8

A 
           0  1     2 3
     ------------------
 0  |      0  1     0 0
 1  |      1  0     0 0
 2  |    0.5 -2     1 0
 3  |  0.625  0 -1.25 1

MA 
       0 1 2     3    4
     ------------------
 0  |  2 1 0     5    4
 1  |  0 2 4     2    8
 2  |  0 0 4    -5   -2
 3  |  0 0 0 -1.75 10.5



# 8.4 가우스 소거법을 사용하여 행렬-벡터 방정식 풀기 

행렬-벡터 방정식의 해을 구하고자 한다고 해보자.

$Ax=b$

MA가 사다리꼴 행렬 U가 되는 그러한 행렬 M을 계산하자. 그리고, 위의 식 양변에 M을 곱하면 다음을 얻는다.

$MAx=Mb$

새로운 방정식은 MA가 사다리꼴이므로 원래 방정식보다 해를 구하기 더 쉽다.

## 8.4.1 행렬이 사다리꼴일 때 행렬-벡터 방정식의 해 구하기-가역적인 경우

U가 사다리꼴 행렬이면서 가역적인 경우, U는 정방행렬이고 그 대각원소들은 영이 아니다.

이것은 상삼각행렬이며 후진대입법을 사용하여 방정식 $Ux=b$를 풀 수 있다.

## 8.4.2 엔트리가 영인 행들에 대한 처리

일반적인 경우를 고려해 보자. U가 삼각행렬이 될 수 없는 경우가 두 가지 있다.
- 엔트리가 모두 영인 행들이 있을 수 있다.
- U의 열들 중 어떠한 행도 그 행의 맨 왼쪽 영이 아닌 엔트리가 이 열에 위치하지 않는 그러한 열들이 있을 수 있다.

첫 번째 이슈에서 엔트리가 영인 행들은 그냥 무시하면 된다. 
- $a_i\cdot x=b_i$에서 $a_i=0$이다.
 - 만약 $b_i=0$이면 방정식은 x 값에 관계없이 항상 성립한다.
 - 만약 $b_i\neq0$이면 방벙식은 x 값에 관계없이 항상 성립하지 않는다.

## 8.4.3 관련없는 열들에 대한 처리

다음 예를 살펴보자.

$\begin{bmatrix} 1 &&  &&1 &&  &&  \\  && 2 &&  && 3 &&  \\  &&  &&  && 1 && 9 \end{bmatrix} * \begin{bmatrix} x_a && x_b &&x_c &&x_d  &&x_e \end{bmatrix} = \begin{bmatrix} 1 \\ 1\\ 1 \end{bmatrix}$

어떤 열의 위치에 영이 아닌 맨 왼쪽 엔트리를 가지는 행이 하나도 없는 경우 그런 열은 버린다. 예제에서는 C, E를 버린 다음 시스템은 아래와 같다.

$\begin{bmatrix} 1 &&  &&   \\  && 2 &&  3  \\  &&  &&  1  \end{bmatrix} * \begin{bmatrix} x_a && x_b  &&x_d   \end{bmatrix} = \begin{bmatrix} 1 \\ 1\\ 1 \end{bmatrix}$

이 시스템은 삼각형 형태이고 후진대입법을 사용하여 풀 수 있다.

버려진 열에 대응하는 변수  $x_c, x_e$는 영으로 설정하다. 행렬-벡터 곱셈의 선형결합 정의를 사용하여 살펴보면 버려진 열들은 선형결하베 아무것도 기여하는 것이 없다. 이것이 보여 주는 것은 이렇게 변수에 값을 할당한 솔루션은 버려진 열들을 다시 채워넣을 경우에도 맞는 솔루션이 된다는 것이다.

## 8.4.4 단순한 인증기법 공격하기 및 개선하기

- $\hat x$: GF(2)상의 n-벡터로 패스워드
- $a$: 컴퓨터는 시도로서의 n-벡터
- $a\cdot \hat x$: 사용자의 응답

이브는 m개 쌍 $a_1,b_1,...,a_m,b_m$에 대해 알고 있으면 아래와 같이 표현할 수 있고, rankA가 n이 되면 솔루션은 유일하게 되고, 이브는 가우스 소거법을 사용하여 그 솔루션을 찾아 패스워드를 얻을 수 있다.

$\begin{bmatrix} a_1 \\ ... \\ a_m \end{bmatrix} \begin{bmatrix} \hat x \end{bmatrix}=\begin{bmatrix} b_1 \\ ... \\ b_m \end{bmatrix}$


**의도적 실수를 도입하여 조금 더 안전하게 만들기**
- 대략 전체 횟수의 1/6정도, 사용자는 랜덤하게 잘못된 도트곱을 전송한다.
- 컴퓨터는 사용자가 75%정도 맞게 응답하면 패스워드를 알고 있다고 생각한다.

사실, 엄청나게 많은 횟수의 시도-응답을 관찰하더라도 솔루션ㅇㄹ 찾는데 사용할 수 있는 알려진 효과적인 알고리즘은 없다. GF(2)상의 규모가 큰 행렬-벡터 방정식에 대한 "근사" 해를 찾는 것은 어려운 계산문제이다.




# 8.5 영공간에 대한 기저 찾기

주어진 행렬 A에 대해, 벡터공간 $\{v:v*A=0\}$에 대한 기저를 찾는 알고리즘을 기술해 보자. 이것은 $A^T$의 영공간이다.  
사다리꼴 행렬이 되도록하는 $MA=U$의 가역적인 M행렬을 찾는 것이다. 벡터-행렬 정의를 사용하기 위해 M과 U를 행들로 구선된 것으로 해석한다.  
$\begin{bmatrix} b_1  \\  ...  \\  b_m  \end{bmatrix} * \begin{bmatrix}   \\&& A && \\ \\   \end{bmatrix} = \begin{bmatrix} u_1 \\ ... \\ u_m \end{bmatrix}$  
만약 $u_i$가 영벡터이면 M의 대응하는 행 $b_i$는 $b_i*A=0$인 성질을 가진다.


### Example 8.5.1 
A는 GF(2)상의 다음 행렬이라 해보자.
$A=\begin{bmatrix} 0 && 0 && 0 && one && 0 \\  0 && 0 && 0 && one && one  \\  one && 0 && 0 && one && 0 \\ one && 0 && 0 && 0 && one \\ one && 0 && 0 && 0 && 0\end{bmatrix}$  
아래는 MA=U를 만족하는 행렬이다.


$\begin{bmatrix} 0 && 0 && one && 0 && 0 \\  one && 0 && 0 && 0 && 0  \\  one && one && 0 && 0 && 0 \\ 0 && one && one && one && 0 \\ one && 0 && one && 0 && 0\end{bmatrix}*\begin{bmatrix} 0 && 0 && 0 && one && 0 \\  0 && 0 && 0 && one && one  \\  one && 0 && 0 && one && 0 \\ one && 0 && 0 && 0 && one \\ one && 0 && 0 && 0 && 0\end{bmatrix}=\begin{bmatrix} one && 0 && 0 && one && 0 \\  0 && 0 && 0 && one && 0  \\  0 && 0 && 0 && 0 && one \\ 0 && 0 && 0 && 0 && 0 \\ 0 && 0 && 0 && 0 && 0\end{bmatrix}$


우변 행렬의행 3과 4는 영벡터이고, 좌변의 첫 번째 행렬 M의 행 3과 4는 벡터공간 $\{v:v*A=0\}$에 속한다. 이들 두 벡터가 사실상 이 벡터공간에 대한 기저를 형성함을 보여 줄 것이다.

___

선택된 M의 행들이 벡터공간 $\{v:v*A=0\}$에 대한 기저라는 것을 보여 주기 위해, 다음 두 가지를 증명해야 한다.
- 선택된 행들은 일차독립이다.
- 선택된 행들은 벡터공간을 생성(Span)한다.


M은 가역행렬이므로 정방행렬이고 그 열들은 일차독립니다. 따라서, 이 행렬의 행들도 일차독립이다. 그러므로, 행들의 임의의 부분집합은 일차독립이다.  
만약 선택된 해들의 랭크가 그 벡터공간의 차원과 동일하다는 것을 보여 줄 수 있으면, Dimension Principle의 Property D2에 의해 선택된 행들의 생성은 사실상 그 벡터공간과 동일하다는 것을 보여 줄 수 있다.  
선택된 행들은 일차독립이므로, 이들의 랭크는 행들의 개수 s와 동일하다. A의 행의 개수를 m이라 하자.  
m=(U의 영이 아닌 행수 개수) + (U의 엔트리가 모두 영인 행의 개수)  
&nbsp;&nbsp; = rankA + s  
Rank-Nullity Theorem(Kernel-Image Theorem의 행렬버전)에 의하면,  
m=rankA+nullity$A^T$  
그러므로, $s=nulltiyA^T$


# 8.6 정수 인수분해

### Theorem 8.6.1 Prime Factorization Theorem
임의의 양의 정수 N에 대해, 곱이 N이 되는 유일한 소수의 그룹이 존재한다.

In [2]:
def prime_factorize(N):
    if is_prime(N):
        return [N]
    a, b = factor(N) # 어려운 것은 factor(N)을 신속하게 실행될 수 있도록 구현하는 것이다.
    return prime_factorize(a) + prime_factorize(b)

## 8.6.1 인수분해에 대한 첫 번째 시도

*나눗셈 시도(trial divisions)*는 특정 정수 b에 대해 N이 b로 나누어지는 테스트하는 것이다. 나눗셈 시도는 단순한 연산이 아니다. 이것은 정확한 산술연산이 이루어져야 하므로 부동소수에 대한 연산보다 훨씬 더 느리다.

In [3]:
def find_divisor(N):
    for i in range(2, N):
        if N % i == 0:
            return i

### Claim
만약 N이 합성수이면, 자명하지 않는 인수는 $\sqrt N$보다 작거나 같다.

___
$\sqrt N$보다 작거나 같은 소수들에 의한 나눗셈 시도만을 고려해보자. Prime Number Theorem에 의하면, 숫자 K보다 작거나 같은 소수는 대략 $K/\ln(K)$이다.

# 8.7 Lab: 임계치 비밀 공유(Threshold Secret-Sharing)

임계치 비밀 공유 기법을 사용해 보자. 이 기법에서는, 예를 들어, 4명의 조교에게 비밀을 분할하고, 임의의 3명의 조교가 모이면 비밀을 풀 수 있지만 임의의 2명으로는 그 비밀을 풀 수 없다.

## 8.7.1 첫 번째 시도

GF(2)상의 5개 3-벡터: $a_0, a_1, a_2, a_3, a_4$가 있다. 이 벡터들은 알려져 있고 다음 요구조건을 만족한다고 해 보자.

### Requirement
모든 세 원소의 집합은 일차독립이다.

___
1-비트 비밀 s를 조교들 사이에 공유한다고 해보자. 랜덤하게 $a_0\cdot u=s$가 되는 3-벡터 u를 선택한다. u는 비밀로 유지하지만, 다른 도트곱을 계산한다.  
$\beta_1=a_1\cdot u$  
$\beta_2=a_2\cdot u$  
$\beta_3=a_3\cdot u$  
$\beta_4=a_4\cdot u$  
각 $\beta$는 TA(조교)들에게 주어진다. 주어진 비트는 그 TA의 몫(share)이라고 한다.

TA1, 2, 3이 비밀을 풀고하 한다고 해보자. 그들은 다음 행렬-벡터 방정식을 푼다.  
$\begin{bmatrix} a_1 \\ --- \\  a_2  \\---\\  a_3  \end{bmatrix} * \begin{bmatrix}   x_1 \\ x_2 \\ x_3   \end{bmatrix} = \begin{bmatrix} \beta_1 \\ \beta_2 \\ \beta_3 \end{bmatrix}$  

$a_1, a_2, a_3$은 일차독립이고, 정방행렬이므로 이 행렬은 가역적이고 그래서 유일한 해가 존재한다. 그러므로, 세명의 TA는 비밀 벡터 u를 얻을 수 있고, $a_0\cdot u$로 비밀 s를 알 수 있다.

이제, 두명의 정직하지 않는 조교들이 2명이 몰래 비밀을 얻으려고 한다고 해보자. 3x3 행렬 식을 만들기 위해 $\beta$가 하나고 부족하고 따라서 유일한 해를 찾을 수 없다.

**위 기법은 동작하는 것처럼 보이지만 문제가 있다. 문제는 요구조건을 만족하는 5개의 3-벡터가 없다는 것이다. 위 기법이 동작할 만큼 충분한 수의 GF(2)상의 3-벡터는 없다.**

## 8.7.2 동작되는 기법

더 큰 벡터들을 고려해 보자. GF(2)상의 10개의 6-벡터 $a_0, b_0, a_1, b_1, a_2, b_2, a_3, b_3, a_4, b_4$를 찾아보자. 이 벡터들으 아래와 같이 5개이 쌍을 구성한다고 생각한다.
- $a_0, b_0$으로 구성되는 쌍 0
- $a_1, b_1$으로 구성되는 쌍 1
- $a_2, b_2$으로 구성되는 쌍 2
- $a_3, b_3$으로 구성되는 쌍 3
- $a_4, b_4$으로 구성되는 쌍 4

### Requirement
임의의 세 쌍에 대해, 대응하는 6개의 벡터들은 일차독립이다.

___
이 기법에서 두 비트 s와 t를 공유하도록 하기 위해, $a_0\cdot u=s, b_0\cdot u=t$를 만족하는 비밀 6-벡터 u를 선택한다. 그 다음에 $\beta, \gamma$를 각 벡터들로 계산하여 TA들에게 주어진다.

### 복원 가능성(Recoverability)
임의의 세 TA가 함께 6x6 행렬을 가진 행렬-벡터 방정식을 풀어 u를 얻을 수 있다.

$\begin{bmatrix} a_1 \\ b_1 \\  a_2  \\ b_2 \\  a_3 \\ b_3  \end{bmatrix} * \begin{bmatrix}  \\ \\ \\ x \\ \\  \\  \end{bmatrix} = \begin{bmatrix} \beta_1 \\ \gamma_1 \\ \beta_2 \\ \gamma_2 \\ \beta_3 \\ \gamma_3  \end{bmatrix}$  

벡터 $a_1, b_1, a_2, b_2, a_3, b_3$은 일차독립이므로, 행렬을 가역적이고, 그래서 이 방정식에 대한 유일한 해가 존재한다.

### 비밀유지
임의의 두 TA에 대해, 6x6 행렬-벡터 방정식을 만들기에는 오른쪽의 $\beta, \gamma$의 한 쌍을 알지 못하기 때문에 유일한 해를 가지지 않는다.

## 8.7.3 구현하기

In [12]:
from vecutil import list2vec
from GF2 import one

a0 = list2vec([one, one, 0, one, 0, one])
b0 = list2vec([one, one, 0, 0, 0, one])

## 8.7.4 u 생성하기

### Task 8.7.1
다음 스펙을 가지는 프로시저, choose_secret_vector(s, t)를 작성해 보자.
- input
 - GF(2)필드 원소 s
 - GF(2)필드 원소 t
- output
 - $a_0\cdot u=s$와 $b_0\cdot u =t$를 만족하는 랜덤 6-벡터 u

In [13]:
import random

def randGF2(): return random.randint(0, 1)*one

def choose_secret_vector(s, t):
    u = list2vec([randGF2() for i in range(len(a0.D))])
    while a0*u != s or b0*u != t:
        u = list2vec([randGF2() for i in range(len(a0.D))])
    return u

u = choose_secret_vector(one, one)
print('u:', u)
print('a0*u:', a0*u)
print('b0*u:', b0*u)

u: 
 0 1 2 3   4   5
----------------
 0 0 0 0 one one
a0*u: one
b0*u: one


## 8.7.5 요구조건을 만족하는 벡터 찾기

### Task 8.7.2
아래 요구조건이 만족되는 GF(2)상의 벡터들 $a_1, b_1, a_2, b_2, a_3, b_3, a_4, b_4$를 선택해 보자.
- 임의의 세 쌍에 대해, 대응하는 여섯 개의 벡터들은 일차독립니다.

In [48]:
from independence import rank
import itertools
from matutil import rowdict2mat

while True:
    L = [(a0, b0)]
    for _ in range(3):
        a = list2vec([randGF2() for i in range(len(a0.D))])
        b = list2vec([randGF2() for i in range(len(a0.D))])
        L.append((a, b))
    
    independence = True
    for Ls in itertools.combinations(L, 3):
        _L = [v for L in Ls for v in L]
        if rank(_L) != len(_L):
            independence = False

    if independence:
        break
L = [vec for pair in L for vec in pair]     
L = rowdict2mat(L)
print('L', L)

L 
         0   1   2   3   4   5
     -------------------------
 0  |  one one   0 one   0 one
 1  |  one one   0   0   0 one
 2  |    0 one   0 one   0 one
 3  |  one one one   0 one one
 4  |    0 one   0   0 one one
 5  |    0   0 one one   0 one
 6  |    0   0   0   0 one one
 7  |    0   0   0   0   0 one



## 8.7.6 문자열 공유하기

In [49]:
from matutil import coldict2mat
from bitutil import str2bits
from bitutil import bits2str
from bitutil import bits2mat
from bitutil import mat2bits

bits = str2bits('Rosebud')
bitmat = bits2mat(bits, 2)
print('bit2mat', bitmat)

u_list = []
for col in bitmat.D[1]:
    u_list.append(choose_secret_vector(bitmat[0, col], bitmat[1, col]))
    
U = coldict2mat(u_list)
print('\nU', U)

share = L*U
print('\nshare', share)

bit2mat 
         0 1  10  11  12  13  14  15  16 17  18  19   2  20  21  22  23 24  25  26  27   3   4   5   6   7   8 9
     -----------------------------------------------------------------------------------------------------------
 0  |    0 0 one one one one   0 one   0  0   0 one one one one one one  0 one   0 one one one one   0 one one 0
 1  |  one 0 one   0   0   0 one   0 one  0 one   0   0   0   0 one   0  0   0 one   0   0 one one one   0 one 0


U 
         0   1  10  11  12  13  14  15  16  17  18  19   2  20  21  22  23  24  25  26  27   3   4   5   6   7   8   9
     -----------------------------------------------------------------------------------------------------------------
 0  |  one one   0 one one   0   0 one   0 one one   0 one   0 one   0   0 one   0   0 one   0 one   0 one   0 one   0
 1  |    0 one   0   0 one   0   0   0 one   0 one   0   0   0 one   0   0   0 one   0 one one   0   0 one   0   0 one
 2  |    0 one one   0 one one one one one one   0   0 one

# 8.8 Lab: 정수를 인수분해하기

## 8.8.1 제곱근을 사용한 첫 번째 시도

현대적인 인수분해 알고리즘을 향한 첫 단계에서, 다음을 만족하는 정수 a와 b를 찾는다고 해 보자.

$a^2-b^2=N$

그러면, 다음이 얻어진다.

$(a-b)(a+b)=N$

그래서, a-b와 a+b는 N의 약수(인수)이다. 우리가 기대하는 것은 이들이 자명하지 않는 약수이길 원한다(즉, a-b는 1도 아니고 N도 아니다)

### Task 8.8.1
$a^2-b^2=N$을 만족하는 정수 a와 b를 찾기 위해, 다음 알고리즘을 구현하는 프로시저, root_method(N)을 작성해 보자.

In [89]:
from factoring_support import intsqrt

def root_method(N):
    a = intsqrt(N)+1
    while a < N:
        divisor = intsqrt(a**2-N)
        if divisor**2 == a**2-N:
            return a - divisor
        else:
            a += 1
    return 1
            
print('55:', root_method(55))
print('77:', root_method(77))
print('146771:', root_method(146771))
print('118:', root_method(118))

55: 5
77: 1
146771: 317
118: 1


## 8.8.2 최대 공약수에 대한 유클리드 알고리즘

### Task 8.8.2
gcd에 대한 코드를 시험해 보자. 정수 r,s,t가 있고, a=r*s, b=t*s로 설정하고 a와 b의 최대공약수 d를 찾는다. d는 다음 성질을 가지는 것을 확인해 보자.
- a는 d에 의해 나누어질 수 있다.
- b는 d에 의해 나누어질 수 있다.
- $d\geq s$

In [93]:
def gcd(x, y): return x if y==0 else gcd(y, x%y)

r = 2490
s = 89
t = 3509

a = r*s
b = t*s

d = gcd(a, b)

print('a:', a)
print('b:', b)
print('d:', d)
print('a % d:', a % d)
print('b % d:', b % d)
print('d, s:', d, s)

a: 221610
b: 312301
d: 89
a % d: 0
b % d: 0
d, s: 89 89


## 8.8.3 제곱근 사용하기 - 다시 보기

$a^2-b^2=N$을 만족하는 그런 정수 a, b를 찾는 것은 너무 어렵다. 조건을 완화하야 $a^2-b^2$이 N에 의해 나누어질 수 있는 나누어질 수 있는 그런 정수 a와 b를 찾아볼 것이다. 그러면, 다음을 만족하는 또 다른 정수 k가 있다.

$a^2-b^2=kN$

위 식은 다음과 같이 쓸 수 있다.  


$(a-b)(a+b)=kN$

곱이 kN인 소수들의 그룹(bag of primes) 내 모든 소수는 아래를 만족한다.
- 곱이 k인 소수들의 그룹 또는 곱이 N인 소수들의 그룹 둘 중 어느 하나에 속하고,
- 곱이 a-b인 소수들의 그룹 또는 곱이 a+b인 소수들의 그룹 둘 중 어느 하나에 속한다.

### Task 8.8.3
$a*a-b*b$ 는 N에 의해 나누어짐을 입증해 보자. 이것은 a-b와 N의 최대공약수는 N의 자명하지 않는 약수일 가능성이 있음을 의미한다.

In [96]:
N = 367160330145890434494322103
a = 67469780066325164
b = 9429601150488992

print('a^2-b^2 % N:', (a**2-b**2)%N)
print('gcd(a-b, N):', gcd(a-b, N))

a^2-b^2 % N: 0
gcd(a-b, N): 19117318483477


### Task 8.8.4
dumb_factor(x, primeset)을 시험해 보자.

In [100]:
from factoring_support import dumb_factor

primeset = {2,3,5,7,11,13}

print('12:', dumb_factor(12, primeset))
print('154:', dumb_factor(154, primeset))
print('2*3*3*3*11*11*13:', dumb_factor(2*3*3*3*11*11*13, primeset))
print('2*17:', dumb_factor(2*17, primeset))
print('2*3*5*6*19:', dumb_factor(2*3*5*6*19, primeset))

12: [(2, 2), (3, 1)]
154: [(2, 1), (7, 1), (11, 1)]
2*3*3*3*11*11*13: [(2, 1), (3, 3), (11, 2), (13, 1)]
2*17: []
2*3*5*6*19: []


### Task 8.8.5
주어진 정주 i에 대해 만약 i가 홀수이면 one을, 만약 i가 짝수이면 0을 리턴하는 프로시저, int2GF2(i)를 작성해 보자.

In [103]:
from GF2 import one

def int2GF2(i):
    return 0 if i%2==0 else one

print('3:', int2GF2(3))
print('4:', int2GF2(4))

3: one
4: 0


### Task 8.8.6
make_Vec(primeset, factors)을 작성해 보자.
- input
 - 소수들의 집합 primeset
 - 리스트 $factors=[(p_1,a_1),(p_2,a_2),...,(p_s,a_s)]$ $p_i$는 primeset에 속한다.
- output
 - 정의역 primeset을 가지는 GF(2)상의 primeset-벡터 v. 이때, i=1,...s에 대해, $v[p_i]$=int2GF2($a_i$)을 만족한다.

In [124]:
from vec import Vec

def make_Vec(primeset, factors):
    v = Vec(primeset, {})
    for f in factors:
        v[f[0]] = int2GF2(f[1])
    return v

print('{2,3,5,7,11}, [(3,1)]:', make_Vec({2,3,5,7,11}, [(3,1)]))
print('{2,3,5,7,11}, [(2,17),(3,0),(5,1),(11,3)]:', make_Vec({2,3,5,7,11}, [(2,17),(3,0),(5,1),(11,3)]))

{2,3,5,7,11}, [(3,1)]: 
 11 2   3 5 7
-------------
  0 0 one 0 0
{2,3,5,7,11}, [(2,17),(3,0),(5,1),(11,3)]: 
  11   2 3   5 7
----------------
 one one 0 one 0


### Task 8.8.7
프로시저 find_candidates(N, primeset)을 작성해 보자.
- input
 - 인수분해할 N
 - 소수들의 집합 primeset
- ouput
 - $a_0,a_1,a_2,...$ 로 구성되는 리스트 roots. 이때, $a_i\cdot a_i - N$은 primeset상에서 완전히 인수분해 될 수 있다.
 - 리스트 rowlist. 이 리스트이 원소 i는 $a_i$에 대응하는 GF(2)상의 primeset-벡터이다.

In [209]:
from factoring_support import primes

def find_candidates(N, primeset):
    roots = []
    rowlist = []
    x = intsqrt(N)+2
    while len(roots) < len(primeset)+1:
        factors = dumb_factor(x*x - N, primeset)
        if factors:
            roots.append(x)
            rowlist.append(make_Vec(primeset, factors))
        x += 1
    return roots, rowlist

N = 2419
roots, rowlist = find_candidates(N, primes(32)) 
print('roots:', roots)
print('rowlist:') 
rowlist

roots: [51, 52, 53, 58, 61, 62, 63, 67, 68, 71, 77, 79]
rowlist:


[Vec({2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31},{2: one, 7: one, 13: one}),
 Vec({2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31},{3: one, 5: one, 19: one}),
 Vec({2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31},{2: one, 3: one, 5: one, 13: one}),
 Vec({2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31},{3: one, 5: one, 7: one}),
 Vec({2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31},{2: one, 3: one, 7: one, 31: one}),
 Vec({2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31},{3: one, 5: 0, 19: one}),
 Vec({2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31},{2: one, 5: 0, 31: one}),
 Vec({2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31},{2: one, 3: 0, 5: one, 23: one}),
 Vec({2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31},{3: 0, 5: one, 7: 0}),
 Vec({2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31},{2: one, 3: one, 19: one, 23: one}),
 Vec({2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31},{2: one, 3: one, 5: one, 13: one}),
 Vec({2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31},{2: one, 3: one, 7: 0, 13: one})]

### Task 8.8.8 
약수를 찾기 위해, a=$53\cdot 77$, b=$2\cdot 3^3\cdot 5 \cdot 13$이라 하고 gcd(a-b, N)을 계산해 보자. 적합한 N의 약수를 찾았는가?

In [140]:
a = 53*77
b = 2*3*3*5*13
N = 2419
gcd(a-b, N)

41

### Task 8.8.9
다시, $a=52\cdot67\cdot71, b=2\cdot3^2\cdot5\cdot19\cdot23$이라 하고 gcd(a-b,N)을 계산해 보자. 적합한 N의 약수를 찾았는가?

In [143]:
a = 52*67*71
b = 2*3*3*5*19*23
gcd(a-b, N)

2419

___
완전 제곱이 되는 행들의 결합을 어떻게 알았는가? 여기에 선형대수가 사용된다. 이러한 행들에 있는 벡터들의 합은 영벡터이다. 이는 $v*A$의 영벡터가 되는 그러한 영이 아닌 벡터 v를 찾는 것과 같다.  
이러한 벡터가 존재하는지 어덯게 아는가? rowlist 내의 각 벡터는 preimeset-벡터이고 그래서 차원이 K=len(primelist)이다. 그러므로, 이러한 벡터들의 랭크는 최대 K이지만 rowlist는 적어도 K+1벡터들로 구성된다. 따라서, 행들은 일차종속이다. 이러한 벡터는 가우스 소거법을 사용하여 행렬을 사다리꼴로 변형할 때 적어도 하나 이상은 반드시 영이 된다.

### Task 8.8.10
프로시저, find_a_and_b(v, roots, N)을 작성해 보자.
- input
 - 벡터 v(M의 행들 중 하나)
 - 리스트 roots
 - 인수분해할 정수 N
- ouput
 - a: 벡터 v의 영이 아닌 엔트리들에 대응하는 roots의 원소들로 이루어진 리스트 alist의 원소들의 곱
 - b: intsqrt($x\cdot x-N:x\in alist$)

In [210]:
from echelon import transformation_rows
from factoring_support import prod

def find_a_and_b(v, roots, N):
    alist = [roots[i] for i, d in enumerate(v.D) if v[d] is not 0]
    a = prod(alist)
    c = prod([a*a-N for a in alist])
    b = intsqrt(c)
    
    assert b*b == c
    return a, b

A = rowdict2mat(rowlist)
M = transformation_rows(rowlist)

a, b = find_a_and_b(M[-1], roots, N)
divisor = gcd(a-b, N)
print('M[-1]*A:', M[-1]*A)
print('a, b:', a, b)
print('diviosr:', divisor)

print('\n')
a, b = find_a_and_b(M[-2], roots, N)
divisor = gcd(a-b, N)
print('M[-1]*A:', M[-2]*A)
print('a, b:', a, b)
print('diviosr:', divisor)

M[-1]*A: 
 11 13 17 19 2 23 29 3 31 5 7
-----------------------------
  0  0  0  0 0  0  0 0  0 0 0
a, b: 13498888 778050
diviosr: 1


M[-1]*A: 
 11 13 17 19 2 23 29 3 31 5 7
-----------------------------
  0  0  0  0 0  0  0 0  0 0 0
a, b: 4081 1170
diviosr: 41


### Task 8.8.11
N=2461799993978700679이라 하고 N을 인수분해 해 보자.

In [222]:
%%time
N = 2461799993978700679
roots, rowlist = find_candidates(N, primes(10000))

M = transformation_rows(rowlist)
for idx, v in enumerate(reversed(M)):
    a, b = find_a_and_b(v, roots, N)
    divisor = gcd(a-b, N)

    if divisor != 1 and divisor != N:
        print('idx:', idx)
        print('divisor:', divisor)
        break

idx: 1
divisor: 1230926561
CPU times: user 1min 58s, sys: 226 ms, total: 1min 58s
Wall time: 1min 58s


### Task 8.8.12
더 큰 수인 N=20672783502493917028427이라 하고 N을 인수분해 해 보자.

In [228]:
%%time
N = 20672783502493917028427
roots, rowlist = find_candidates(N, primes(10000))

A = rowdict2mat(rowlist)
M = transformation_rows(rowlist)
for idx, v in enumerate(reversed(M)):
    a, b = find_a_and_b(v, roots, N)
    divisor = gcd(a-b, N)
    
    if divisor != 1 and divisor != N:
        print('idx, v:', idx)
        print('divisor:', divisor)
        break

idx, v: 5
divisor: 1230926561
CPU times: user 8min 7s, sys: 581 ms, total: 8min 8s
Wall time: 8min 8s


### Task 8.8.13
큰 소수는 작은 소수보다 정수의 인수분해에 포함될 가능성이 더 낮다라는 힌트를 이용하여 M을 찾는 데 걸리는 시간을 줄여보자.

In [229]:
%%time
N = 20672783502493917028427
roots, rowlist = find_candidates(N, primes(10000))

A = rowdict2mat(rowlist)
M = transformation_rows(rowlist, sorted(primes(10000), reverse=True))
for idx, v in enumerate(reversed(M)):
    a, b = find_a_and_b(v, roots, N)
    divisor = gcd(a-b, N)
    
    if divisor != 1 and divisor != N:
        print('idx, v:', idx)
        print('divisor:', divisor)
        break

idx, v: 5
divisor: 1230926561
CPU times: user 5min 29s, sys: 307 ms, total: 5min 29s
Wall time: 5min 29s


# 8.9 Review questions

### Q) 사다리꼴(echelon form)은 무엇인가?
- m x n행렬 A는 만약 다음 조건을 만족하면 *사다리꼴*이다. 즉, 임의의 행에 대해, 만약 그 행의 첫 번째 영이 아닌 엔트리가 위치 k에 있으면 그 행 이전의 모든 행의 첫 번째 영이 아닌 엔트리는 k보다 작은 어떤 위치에 있다.

### Q) 사다리꼴로 된 행렬의 랭크에 대해 무엇을 알 수 있는가?
- 사다리꼴로 된 행렬의 랭크는 모든 엔트리가 0인 행을 제외한 행의 수가 랭크이다. 이 사다리꼴로 된 행렬은 원래 행렬의 랭크와 같다
- 행렬 A와 M에 대해, 만약 M이 가역적이면 Row MA=Row A이다.

### Q) 행렬은 어떻게 가역행렬의 곱셈에 의해 사다리꼴로 변환되는가?
- 기본행 덧셈 연산 원리로 사다리꼴로 변환된다.

### Q) 가우스 소거법은 행렬의 영공간에 대한 기저를 찾는 데 어떻게 사용될 수 있는가?
- 가우스 소거법을 이용하면 행렬 A를 사다리꼴 행렬로 만든는 가역 행렬 M을 구할 수 있다. 사다리꼴로 된 행렬에서 모든 엔트리가 0인 행에 대응하는 M의 행은 영공간에 대한 기저를 생성한다.

### Q) 가우스 소거법은 행렬이 가역적일 때 행렬-벡터 방정식을 푸는데 어떻게 사용될 수 있는가?
- 가우스 소거법을 이용하면 행렬 A를 사다리꼴 행렬로 만들 수 있고 사다리꼴 행렬은 후진대입법을 이용하여 쉽게 방정식을 풀 수 있다.

# Problems

### Problem 8.9.1
GF(2)상의 다음 행렬에 대해 가우스 소거법을 손으로 실행해 보자.

In [1]:
###### 주어진 행렬 #####
#    A   B   C   D
# 0 one one  0   0
# 1 one  0  one  0
# 2  0  one one one
# 3 one  0   0   0
#####################

# iter=0, pivot=0, 기본행 덧셈: 0+1, 0+3
###################
#    A   B   C   D
# 0 one one  0   0
# 1  0  one one  0
# 2  0  one one one
# 3  0  one  0   0
###################

# iter=1, pivot=1, 기본행 덧셈: 1+2, 1+3
###################
#    A   B   C   D
# 0 one one  0   0
# 1  0  one one  0
# 2  0   0   0 one
# 3  0   0  one  0
###################

# iter=2, 행 교환: 2,3 
###################
#    A   B   C   D
# 0 one one  0   0
# 1  0  one one  0
# 2  0   0  one  0
# 3  0   0   0 one
###################

### Problem 8.9.2
주어진 각 행렬은 거의 사다리꼴 형태이다. MINIMUM 숫자의 원소들의 값을 0으로 바꾸어 사다리꼴 행렬을 얻는다. 행렬의 행 또는 열들의 순서를 바꾸는 것은 허용되지 않는다.

In [2]:
#1. 
# 1    2    0    2    0
# 0    1    0    3    4
# 0    0    2    3    4 
# 1->0 0    0    2    0
# 0    3->0 0    0    4

#2. 
# 0    4    3    4    4
# 6->0 5->0 4    2    0
# 0    0    0    0    1 
# 0    0    0    0    2->0

#3. 
# 1    0    0    1    
# 1->0 0    0    1
# 0    0    0    1->0

#3. 
# 1    0    0    0    
# 0    1    0    0
# 1->0 1->0 0    0
# 0    0    0    1->0

### Problem 8.9.3
프로시저 is_echelon(A)를 작성해 보자.
- input
 - 행렬 A
- ouput
 - 행렬 A가 사다리꼴이면 True, 그렇지 않으면 False

In [6]:
from matutil import listlist2mat

def is_echelon(A):
    echelon = True
    nonzero = -1
    for r in range(len(A.D[0])):
        for c in range(len(A.D[1])):
            if A[r,c] != 0:
                break
        if c <= nonzero:
            echelon = False
            break
        else:
            nonzero = c
    return echelon

A = listlist2mat([[2,1,0],[0,-4,0],[0,0,1]])
print('is echelon:', is_echelon(A), A)

A = listlist2mat([[2,1,0],[-4,0,0],[0,0,1]])
print('is echelon:', is_echelon(A), A)

A = listlist2mat([[2,1,0],[0,3,0],[1,0,1]])
print('is echelon:', is_echelon(A), A)

A = listlist2mat([[1,1,1,1,1],[0,2,0,1,3],[0,0,0,5,3]])
print('is echelon:', is_echelon(A), A)

is echelon: True 
       0  1 2
     --------
 0  |  2  1 0
 1  |  0 -4 0
 2  |  0  0 1

is echelon: False 
        0 1 2
     --------
 0  |   2 1 0
 1  |  -4 0 0
 2  |   0 0 1

is echelon: False 
       0 1 2
     -------
 0  |  2 1 0
 1  |  0 3 0
 2  |  1 0 1

is echelon: True 
       0 1 2 3 4
     -----------
 0  |  1 1 1 1 1
 1  |  0 2 0 1 3
 2  |  0 0 0 5 3



### Problem 8.9.4
다음의 각 행렬-벡터 방정식에 대한 해를 구하여라.

In [7]:
from vecutil import zero_vec
from vecutil import list2vec
from matutil import mat2rowdict

def echelon_solve1(A, b):
    rowlist = mat2rowdict(A)
    D = rowlist[0].D
    n = len(D)
    x = zero_vec(D)
    for r in reversed(range(len(rowlist))):
        c = next((c for c in range(n) if rowlist[r][c]), None)
        if c is not None:
            x[c] = (b[r] - rowlist[r] * x)/rowlist[r][c]            
    return x

A = listlist2mat([[10,2,-3,53],[0,0,1,2013]])
b = list2vec([1,3])
x = echelon_solve1(A, b)
print('A', A)
print('b', b)
print('x', x)

A = listlist2mat([[2,0,1,3],[0,0,5,3],[0,0,0,1]])
b = list2vec([1,-1,3])
x = echelon_solve1(A, b)
print('A', A)
print('b', b)
print('x', x)

A = listlist2mat([[2,2,4,3,2,],[0,0,-1,11,1],[0,0,0,0,5]])
b = list2vec([2,0,10])
x = echelon_solve1(A, b)
print('A', A)
print('b', b)
print('x', x)

A 
        0 1  2        3
     ------------------
 0  |  10 2 -3       53
 1  |   0 0  1 2.01E+03

b 
 0 1
----
 1 3
x 
 0 1 2 3
--------
 1 0 3 0
A 
       0 1 2 3
     ---------
 0  |  2 0 1 3
 1  |  0 0 5 3
 2  |  0 0 0 1

b 
 0  1 2
-------
 1 -1 3
x 
  0 1  2 3
----------
 -3 0 -2 3
A 
       0 1  2  3 4
     -------------
 0  |  2 2  4  3 2
 1  |  0 0 -1 11 1
 2  |  0 0  0  0 5

b 
 0 1  2
-------
 2 0 10
x 
  0 1 2 3 4
-----------
 -5 0 2 0 2


### Problem 8.9.5
다음의 각 행렬-벡터 방정식에 대해, 방정식이 해를 가지는지 알아보자. 만약 해를 가지는 경우, 그 해를 계산하여라.

In [8]:
def echelon_solve2(A, b):
    rowlist = mat2rowdict(A)
    D = rowlist[0].D
    n = len(D)
    x = zero_vec(D)
    for r in reversed(range(len(rowlist))):
        c = next((c for c in range(n) if rowlist[r][c]), None)
        if c is not None:
            x[c] = (b[r] - rowlist[r] * x)/rowlist[r][c]
        elif rowlist[r] == zero_vec(D) and b[r] is not 0:
            return 'Don`t have the solution'
            
    return x


A = listlist2mat([[1,3,-2,1,0],[0,0,2,-3,0],[0,0,0,0,0]])
b = list2vec([5,3,2])
x = echelon_solve2(A, b)
print('A', A)
print('b', b)
print('x', x)

A = listlist2mat([[1,2,-8,-4,0],[0,0,2,12,0],[0,0,0,0,0],[0,0,0,0,0]])
b = list2vec([5,4,0,0])
x = echelon_solve2(A, b)
print('A', A)
print('b', b)
print('x', x)

A 
       0 1  2  3 4
     -------------
 0  |  1 3 -2  1 0
 1  |  0 0  2 -3 0
 2  |  0 0  0  0 0

b 
 0 1 2
------
 5 3 2
x Don`t have the solution
A 
       0 1  2  3 4
     -------------
 0  |  1 2 -8 -4 0
 1  |  0 0  2 12 0
 2  |  0 0  0  0 0
 3  |  0 0  0  0 0

b 
 0 1 2 3
--------
 5 4 0 0
x 
  0 1 2 3 4
-----------
 21 0 2 0 0


### Problem 8.9.6
다음 스펙을 가지는 프로시저, echelon_solver(rowlist, label_list, b)을 제시하여라.
- input
 - rowlist: 정수 n에 대해, n 벡터들의 리스트 rowlist에 의해 표현된 사다리꼴 형태의 행렬
 - label_list: 행렬의 열들의 순서를 제공하는 열-라벨들의 리스트
 - b: 필드 원소들로 구성된 길이가 n인 리스트 b
- output
 - 벡터 x. i=0,1,...,n-1에 대해, rowlist[i]와 x의 도트곱은 만약 rowlist[i]가 영벡터가 아니면 b[i]와 동일하다.

In [22]:
from vec import Vec
from GF2 import one
from matutil import rowdict2mat

def echelon_solve(rowlist, label_list, b):
    D = rowlist[0].D
    x = zero_vec(set(label_list))
    for r in reversed(range(len(rowlist))):
        c = next((c for c in label_list if rowlist[r][c]), None)
        if c is not None:
            x[c] = (b[r] - rowlist[r] * x)/rowlist[r][c]
        elif rowlist[r] == zero_vec(D) and b[r] is not 0:
            return 'Don`t have the solution'
    return x

label_list = ['A', 'B', 'C', 'D', 'E']
rowlist = [Vec(set(label_list), {'A':one, 'B':0, 'C':one, 'D':one, 'E': 0}),
           Vec(set(label_list), {'A':0, 'B':one, 'C':0, 'D':0, 'E': one}),
           Vec(set(label_list), {'A':0, 'B':0, 'C':one, 'D':0, 'E': one}),
           Vec(set(label_list), {'A':0, 'B':0, 'C':0, 'D':0, 'E': one})]
b = [one, 0, one, one]
x = echelon_solve(rowlist, label_list, b)
print('rowlist', rowdict2mat(rowlist))
print('b', b)
print('x', x)

rowlist = [Vec(set(label_list), {'A':one, 'B':one, 'C':0, 'D':one, 'E': 0}),
           Vec(set(label_list), {'A':0, 'B':one, 'C':0, 'D':one, 'E': one}),
           Vec(set(label_list), {'A':0, 'B':0, 'C':one, 'D':0, 'E': one}),
           Vec(set(label_list), {'A':0, 'B':0, 'C':0, 'D':0, 'E': 0})]
b = [one, 0, one, 0]
x = echelon_solve(rowlist, label_list, b)
print('rowlist', rowdict2mat(rowlist))
print('b', b)
print('x', x)

rowlist 
         A   B   C   D   E
     ---------------------
 0  |  one   0 one one   0
 1  |    0 one   0   0 one
 2  |    0   0 one   0 one
 3  |    0   0   0   0 one

b [one, 0, one, one]
x 
   A   B C D   E
----------------
 one one 0 0 one
rowlist 
         A   B   C   D   E
     ---------------------
 0  |  one one   0 one   0
 1  |    0 one   0 one one
 2  |    0   0 one   0 one
 3  |    0   0   0   0   0

b [one, 0, one, 0]
x 
   A B   C D E
--------------
 one 0 one 0 0


### Problem 8.9.7
일반적인 행렬-벡터 방정식을 푸는 프로시저에 대해 알아보자.

In [35]:
from mat import Mat
import echelon


def solve(A, b):
    M = echelon.transformation(A)
    U = M*A
    col_label_list = sorted(A.D[1])
    U_row_dict = mat2rowdict(U)
    rowdict = [U_row_dict[i] for i in U_row_dict]
    return echelon_solve(rowdict, col_label_list, M*b)


A = Mat(({'a','b','c','d'},{'A','B','C','D'}), {('a','A'):one, ('a','B'):one, ('a','C'):0, ('a','D'):one,
                                                ('b','A'):one, ('b','B'):0, ('b','C'):0, ('b','D'):one,
                                                ('c','A'):one, ('c','B'):one, ('c','C'):one, ('c','D'):one,
                                                ('d','A'):0, ('d','B'):0, ('d','C'):one, ('d','D'):one})
g = Vec({'a','b','c','d'}, {'a':one, 'b':0, 'c': one, 'd': 0})
x = solve(A, g)

print('A', A)
print('g', g)
print('x', x)
print('A*x', A*x)

A 
         A   B   C   D
     -----------------
 a  |  one one   0 one
 b  |  one   0   0 one
 c  |  one one one one
 d  |    0   0 one one

g 
   a b   c d
------------
 one 0 one 0
x 
 A   B C D
----------
 0 one 0 0
A*x 
   a b   c d
------------
 one 0 one 0


### Problem 8.9.8
GF(2)상의 행렬들에서 $\{u:u*A=0\}=A^T$에 대한 기저를 찾아보자.

In [51]:
def find_null_space_basis(A):
    M = echelon.transformation(A)
    MA = M*A
    MA_dict = mat2rowdict(MA)
    M_dict = mat2rowdict(M)
    u_list = [M_dict[k] for k, v in MA_dict.items() if v == zero_vec(A.D[1])]
    return u_list
    

A = Mat(({'a','b','c','d','e'},{'A','B','C','D','E'}), {('a','A'):0, ('a','B'):0, ('a','C'):0, ('a','D'):one, ('a','E'):0,
                                                        ('b','A'):0, ('b','B'):0, ('b','C'):0, ('b','D'):one, ('b','E'):one, 
                                                        ('c','A'):one, ('c','B'):0, ('c','C'):0, ('c','D'):one, ('c','E'):0,
                                                        ('d','A'):one, ('d','B'):0, ('d','C'):0, ('d','D'):0, ('d','E'):one,
                                                        ('e','A'):one, ('e','B'):0, ('e','C'):0, ('e','D'):0, ('e','E'):0,}) 

u_list = find_null_space_basis(A)
print('A', A)
print('u', u_list)

A 
         A B C   D   E
     -----------------
 a  |    0 0 0 one   0
 b  |    0 0 0 one one
 c  |  one 0 0 one   0
 d  |  one 0 0   0 one
 e  |  one 0 0   0   0

u [Vec({'d', 'c', 'e', 'b', 'a'},{'d': one, 'c': one, 'e': 0, 'b': one, 'a': 0}), Vec({'d', 'c', 'e', 'b', 'a'},{'d': 0, 'c': one, 'e': one, 'b': 0, 'a': one})]


### Problem 8.9.9
GF(2)상의 행렬들에서 $\{u:u*A=0\}=A^T$에 대한 기저를 찾아보자.

In [52]:
A = Mat(({'a','b','c','d','e'},{'A','B','C','D','E'}), {('a','A'):0, ('a','B'):0, ('a','C'):0, ('a','D'):one, ('a','E'):0,
                                                        ('b','A'):0, ('b','B'):0, ('b','C'):0, ('b','D'):one, ('b','E'):one, 
                                                        ('c','A'):one, ('c','B'):0, ('c','C'):0, ('c','D'):one, ('c','E'):0,
                                                        ('d','A'):one, ('d','B'):one, ('d','C'):one, ('d','D'):0, ('d','E'):one,
                                                        ('e','A'):one, ('e','B'):0, ('e','C'):0, ('e','D'):one, ('e','E'):0,}) 

u_list = find_null_space_basis(A)
print('A', A)
print('u', u_list)

A 
         A   B   C   D   E
     ---------------------
 a  |    0   0   0 one   0
 b  |    0   0   0 one one
 c  |  one   0   0 one   0
 d  |  one one one   0 one
 e  |  one   0   0 one   0

u [Vec({'d', 'c', 'e', 'b', 'a'},{'d': 0, 'c': one, 'e': one, 'b': 0, 'a': 0})]
